## 使用大模型生成文本

LLMs，即大型语言模型，是文本生成背后的关键组件。简而言之，它们由大型预训练的Transformer模型组成，经过训练以预测给定一些输入文本时的下一个单词（或更准确地说，标记）。由于它们一次预测一个标记，因此您需要做一些更复杂的事情来生成新的句子，而不仅仅是调用模型 — 您需要进行自回归生成。

自回归生成是在推断时通过反复调用模型以其自己生成的输出来进行的过程，给定一些初始输入。在🤗 Transformers中，这由generate()方法处理，适用于所有具有生成能力的模型。

本教程将向您展示如何：

- 使用LLM生成文本
- 避免常见陷阱
- 下一步，帮助您充分利用LLM

在开始之前，请确保已安装所有必要的库：

```
pip install transformers bitsandbytes>=0.39.0 -q
```

## 生成文本


为因果语言建模训练的语言模型将文本标记序列作为输入，并返回下一个标记的概率分布。


<video id="video" controls="" preload="none" poster="封面">
      <source id="mp4" src="http://stream.rarelimiting.com/gif_1_1080p.mov" type="video/mp4">
</videos>

LLM自回归生成的一个关键方面是如何从概率分布中选择下一个标记。在这一步中，任何方法都可以，只要最终得到下一次迭代的标记。这意味着可以简单地从概率分布中选择最可能的标记，也可以在从结果分布中抽样之前应用一系列复杂的转换。

<video id="video" controls="" preload="none" poster="封面">
      <source id="mp4" src="http://stream.rarelimiting.com/gif_2_1080p.mov" type="video/mp4">
</videos>


上述过程会重复进行，直到达到某个停止条件。理想情况下，停止条件由模型决定，模型应学会何时输出一个序列结束（EOS）标记。如果不是这种情况，生成会在达到预定义的最大长度时停止。

正确设置标记选择步骤和停止条件对使模型按预期行为至关重要。这就是为什么我们为每个模型关联一个GenerationConfig文件，其中包含良好的默认生成参数设置，并与您的模型一起加载。

让我们来谈谈代码！

>> 如果您对基本LLM用法感兴趣，我们的高级Pipeline接口是一个很好的起点。然而，LLM通常需要高级功能，如量化和对标记选择步骤的精细控制，最好通过generate()来实现。LLM的自回归生成也需要大量资源，并且应在GPU上执行以获得足够的吞吐量。

首先，您需要加载模型。

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1", device_map="auto", load_in_4bit=True
)

您会注意到from_pretrained调用中有两个标志：

- device_map 确保模型被移动到您的GPU
- load_in_4bit 应用4位动态量化以大幅减少资源需求

还有其他初始化模型的方法，但这是一个很好的基准，用于开始使用LLM。

接下来，您需要使用分词器预处理文本输入。

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", padding_side="left")
model_inputs = tokenizer(["A list of colors: red, blue"], return_tensors="pt").to("cuda")

model_inputs变量保存了分词后的文本输入，以及注意力掩码。虽然generate()会尽力推断注意力掩码，但我们建议尽可能传递它以获得最佳结果。

在对输入进行分词后，您可以调用generate()方法返回生成的标记。然后应将生成的标记转换为文本后打印。

In [ ]:
generated_ids = model.generate(**model_inputs)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

最后，您不需要一次处理一个序列！您可以批量处理输入，这将大大提高吞吐量，同时延迟和内存成本较小。您只需要确保正确填充您的输入即可（下面会详细介绍）。

In [ ]:
tokenizer.pad_token = tokenizer.eos_token  # Most LLMs don't have a pad token by default
model_inputs = tokenizer(
    ["A list of colors: red, blue", "Portugal is"], return_tensors="pt", padding=True
).to("cuda")
generated_ids = model.generate(**model_inputs)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

就是这样！几行代码就可以利用LLM的强大功能。

## 常见陷阱

有许多生成策略，有时默认值可能不适合您的用例。如果您的输出与您的预期不一致，我们已经创建了一个常见陷阱列表以及如何避免它们。

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer.pad_token = tokenizer.eos_token  # Most LLMs don't have a pad token by default
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1", device_map="auto", load_in_4bit=True
)

## 生成的输出过短/过长

如果在GenerationConfig文件中未指定，generate默认返回最多20个标记。我们强烈建议在generate调用中手动设置max_new_tokens，以控制它可以返回的最大新标记数。请记住，LLM（更准确地说，仅解码器模型）还会将输入提示作为输出的一部分返回。

In [ ]:
model_inputs = tokenizer(["A sequence of numbers: 1, 2"], return_tensors="pt").to("cuda")

# By default, the output will contain up to 20 tokens
generated_ids = model.generate(**model_inputs)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

# Setting `max_new_tokens` allows you to control the maximum length
generated_ids = model.generate(**model_inputs, max_new_tokens=50)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

### 错误的生成模式

默认情况下，除非在GenerationConfig文件中指定，generate在每次迭代中选择最可能的标记（贪婪解码）。根据您的任务，这可能是不可取的；像聊天机器人或写作文章这样的创造性任务受益于抽样。另一方面，像音频转录或翻译这样的输入驱动任务受益于贪婪解码。通过设置do_sample=True启用抽样，您可以在这篇博文中了解更多关于这个主题的内容。

In [ ]:
# Set seed or reproducibility -- you don't need this unless you want full reproducibility
from transformers import set_seed
set_seed(42)

model_inputs = tokenizer(["I am a cat."], return_tensors="pt").to("cuda")

# LLM + greedy decoding = repetitive, boring output
generated_ids = model.generate(**model_inputs)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

# With sampling, the output becomes more creative!
generated_ids = model.generate(**model_inputs, do_sample=True)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

### 错误的填充方式

LLM是仅解码器架构，意味着它们会继续迭代您的输入提示。如果您的输入长度不相同，它们需要被填充。由于LLM没有经过训练以从填充标记继续，因此您的输入需要进行左填充。确保您还记得传递注意力掩码给generate！

In [ ]:
# The tokenizer initialized above has right-padding active by default: the 1st sequence,
# which is shorter, has padding on the right side. Generation fails to capture the logic.
model_inputs = tokenizer(
    ["1, 2, 3", "A, B, C, D, E"], padding=True, return_tensors="pt"
).to("cuda")
generated_ids = model.generate(**model_inputs)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

# With left-padding, it works as expected!
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", padding_side="left")
tokenizer.pad_token = tokenizer.eos_token  # Most LLMs don't have a pad token by default
model_inputs = tokenizer(
    ["1, 2, 3", "A, B, C, D, E"], padding=True, return_tensors="pt"
).to("cuda")
generated_ids = model.generate(**model_inputs)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

### 错误的提示

一些模型和任务期望特定的输入提示格式才能正常工作。如果未应用此格式，您将会遇到性能下降：模型可能会运行，但效果不如按照预期提示。有关提示的更多信息，包括哪些模型和任务需要注意，可在此指南中找到。让我们看一个使用聊天模板的聊天LLM示例：

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-alpha")
model = AutoModelForCausalLM.from_pretrained(
    "HuggingFaceH4/zephyr-7b-alpha", device_map="auto", load_in_4bit=True
)
set_seed(0)
prompt = """How many helicopters can a human eat in one sitting? Reply as a thug."""
model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
input_length = model_inputs.input_ids.shape[1]
generated_ids = model.generate(**model_inputs, max_new_tokens=20)
print(tokenizer.batch_decode(generated_ids[:, input_length:], skip_special_tokens=True)[0])
# Oh no, it did not follow our instruction to reply as a thug! Let's see what happens when we write
# a better prompt and use the right template for this model (through `tokenizer.apply_chat_template`)

set_seed(0)
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a thug",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]
model_inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to("cuda")
input_length = model_inputs.shape[1]
generated_ids = model.generate(model_inputs, do_sample=True, max_new_tokens=20)
print(tokenizer.batch_decode(generated_ids[:, input_length:], skip_special_tokens=True)[0])
# As we can see, it followed a proper thug style 😎